# Statistical Foundations
### in Python, using pandas, numpy, and scipy

TODO: Motivation

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

## Descriptive statistics: the shape of the data

### Central tendency

mean/median/mode

### Dispersion

SD/variance/range/IQR

## Bivariate analysis: finding similarities in the data

### Correlation

### Grouped comparisons

## Linear regression: relationships between variables

### Simple regression

### 

## Hypothesis testing: making conclusions

### One-Sample t-test

### Two-Sample t-test

### P-values